In [4]:
!pip install -q -U bitsandbytes > /dev/null
!pip install -q -U git+https://github.com/huggingface/transformers.git > /dev/null
!pip install -q -U git+https://github.com/huggingface/peft.git > /dev/null
!pip install -q -U git+https://github.com/huggingface/accelerate.git > /dev/null
!pip install -q datasets > /dev/null

In [5]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, default_data_collator, get_linear_schedule_with_warmup
from peft import get_peft_config, get_peft_model, get_peft_model_state_dict, PrefixTuningConfig, TaskType,PromptTuningInit, PromptTuningConfig
from datasets import load_dataset
from torch.utils.data import DataLoader
from tqdm import tqdm
import os

In [6]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
model_name_or_path = "t5-large"
tokenizer_name_or_path = "t5-large"

text_column = "sentence"
target_column = "built_text"
max_length = 128
lr = 1e-2
num_epochs = 5
batch_size = 8

In [7]:
device

'cuda'

In [8]:
from datasets import load_dataset

dataset = load_dataset("financial_phrasebank",
                       "sentences_allagree")

Generating train split:   0%|          | 0/2264 [00:00<?, ? examples/s]

Dataset financial_phrasebank downloaded and prepared to /root/.cache/huggingface/datasets/financial_phrasebank/sentences_allagree/1.0.0/550bde12e6c30e2674da973a55f57edde5181d53f5a5a34c1531c53f93b7e141. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [9]:
dataset['train'][0]

{'sentence': 'According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .',
 'label': 1}

In [10]:
def text_to_table(data):
  """Returns the text into a table format as shown in
  the example below. The column
    Parameter:
      text(str): A sentence

    Returns:
      table_text:
      less_than_6:
      sent_1
      more_than_6:
      sent_2
  """
  text = data['sentence']
  text_splits = text.split(',')

  built_text = """
less than 6:\n"""

  less_6 = []
  more_6 = []

  for ind, txt in enumerate(text_splits):
    txt = txt.strip()
    snip_len = len(txt.split(' '))

    if snip_len > 6:
      more_6.append(txt)
    else:
      less_6.append(txt)

  for snip in less_6:
    built_text = built_text + f"{snip}\n"

  built_text += '\nmore than 6:\n'

  for snip in more_6:
    built_text = built_text + f"{snip}\n"

  data['built_text'] = built_text

  return data

In [11]:
new_dataset = dataset.map(text_to_table)

Map:   0%|          | 0/2264 [00:00<?, ? examples/s]

In [12]:
for text in new_dataset['train']['built_text'][0:2]:
  print(text)


less than 6:
According to Gran

more than 6:
the company has no plans to move all production to Russia
although that is where the company is growing .


less than 6:
For the last quarter of 2010

more than 6:
Componenta 's net sales doubled to EUR131m from EUR76m for the same period a year earlier
while it moved to a zero pre-tax profit from a pre-tax loss of EUR7m .



In [13]:
new_dataset = new_dataset["train"].train_test_split(test_size=0.1)

new_dataset["validation"] = new_dataset["test"]

del new_dataset["test"]

In [14]:
new_dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'built_text'],
        num_rows: 2037
    })
    validation: Dataset({
        features: ['sentence', 'label', 'built_text'],
        num_rows: 227
    })
})

In [15]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)


def preprocess_function(examples):
    inputs = examples[text_column]
    targets = examples[target_column]
    model_inputs = tokenizer(inputs,
                             max_length=max_length,
                             padding="max_length",
                             truncation=True,
                             return_tensors="pt")
    labels = tokenizer(targets,
                       max_length=max_length,
                       padding="max_length",
                       truncation=True,
                       return_tensors="pt")

    labels = labels["input_ids"]

    labels[labels == tokenizer.pad_token_id] = -100

    model_inputs["labels"] = labels

    return model_inputs

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [16]:
processed_datasets = new_dataset.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=new_dataset["train"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

Running tokenizer on dataset:   0%|          | 0/2037 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/227 [00:00<?, ? examples/s]

In [17]:
processed_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 2037
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 227
    })
})

In [18]:
train_dataset = processed_datasets["train"]
eval_dataset = processed_datasets["validation"]

In [19]:
train_dataloader = DataLoader(train_dataset,
                              shuffle=True,
                              collate_fn=default_data_collator,
                              batch_size=batch_size,
                              pin_memory=True)
eval_dataloader = DataLoader(eval_dataset,
                             collate_fn=default_data_collator,
                             batch_size=batch_size,
                             pin_memory=True)

In [20]:
peft_config = PrefixTuningConfig(task_type=TaskType.SEQ_2_SEQ_LM,
                                 inference_mode=False,
                                 num_virtual_tokens=20)

model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 983,040 || all params: 738,651,136 || trainable%: 0.13308583065659835


In [21]:
import torch

optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataloader) * num_epochs),
)

In [22]:
model.to(device)

PeftModelForSeq2SeqLM(
  (base_model): T5ForConditionalGeneration(
    (shared): Embedding(32128, 1024)
    (encoder): T5Stack(
      (embed_tokens): Embedding(32128, 1024)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=1024, out_features=1024, bias=False)
                (k): Linear(in_features=1024, out_features=1024, bias=False)
                (v): Linear(in_features=1024, out_features=1024, bias=False)
                (o): Linear(in_features=1024, out_features=1024, bias=False)
                (relative_attention_bias): Embedding(32, 16)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseActDense(
                (wi): Linear(in_features=1024, out_features=4096, bias=False)
             

In [23]:
train_dataset[0].items()

dict_items([('input_ids', [505, 932, 2464, 3, 18, 28124, 4400, 5834, 494, 11, 7028, 794, 1002, 13762, 3318, 10536, 411, 63, 354, 41, 454, 3577, 3, 10, 23112, 22480, 3, 61, 243, 469, 41, 505, 932, 2464, 3, 61, 165, 3134, 1453, 5658, 15, 26, 12, 12400, 632, 3, 5, 536, 3, 51, 41, 9513, 632, 3, 5, 2534, 3, 51, 3, 61, 21, 8, 166, 2893, 13, 2464, 45, 12400, 632, 3, 5, 591, 3, 51, 21, 8, 337, 1059, 13, 2628, 3, 5, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), ('attention_mask', [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), ('labels', [705, 145, 431, 10, 72, 145, 431

In [ ]:
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for step, batch in enumerate(tqdm(train_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.detach().float()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    model.eval()
    eval_loss = 0
    eval_preds = []

    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        eval_loss += loss.detach().float()
        eval_preds.extend(
            tokenizer.batch_decode(torch.argmax(outputs.logits, -1).detach().cpu().numpy(), skip_special_tokens=True)
        )

    eval_epoch_loss = eval_loss / len(eval_dataloader)
    eval_ppl = torch.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / len(train_dataloader)
    train_ppl = torch.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")

 58%|█████▊    | 148/255 [01:43<01:14,  1.44it/s]

In [ ]:
correct = 0
total = 0
for pred, true in zip(eval_preds, dataset["validation"]["text_label"]):
    if pred.strip() == true.strip():
        correct += 1
    total += 1
accuracy = correct / total * 100

print(f"{accuracy=} % on the evaluation dataset")

print(f"{eval_preds[:10]=}")

print(f"{dataset['validation']['text_label'][:10]=}")


accuracy=97.79735682819384 % on the evaluation dataset
eval_preds[:10]=['positive', 'neutral', 'neutral', 'positive', 'neutral', 'positive', 'neutral', 'neutral', 'positive', 'positive']
dataset['validation']['text_label'][:10]=['positive', 'neutral', 'neutral', 'positive', 'neutral', 'positive', 'neutral', 'neutral', 'positive', 'neutral']


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
peft_model_id = "Kamaljp/t5-large_PREFIX_TUNING_SEQ2SEQ"

model.push_to_hub("Kamaljp/t5-large_PREFIX_TUNING_SEQ2SEQ", use_auth_token=True)

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.bin:   0%|          | 0.00/3.93M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Kamaljp/t5-large_PREFIX_TUNING_SEQ2SEQ/commit/44b2f94991d5025a83b5ce9edbd68567c1616b7c', commit_message='Upload model', commit_description='', oid='44b2f94991d5025a83b5ce9edbd68567c1616b7c', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
from peft import PeftModel, PeftConfig

peft_model_id = "Kamaljp/t5-large_PREFIX_TUNING_SEQ2SEQ"

config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(model, peft_model_id)

In [ ]:
inputs = tokenizer(
    "The Indian beer market made up 24.41 million liters in January , a rise of 18 percent.",
    return_tensors="pt",
)

In [ ]:
model.to(device)

with torch.no_grad():
    inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = model.generate(input_ids=inputs["input_ids"], max_new_tokens=10)
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True))

['positive']


['positive']

Working on Causal Language modelling

In [ ]:
device = "cuda"
model_name_or_path = "bigscience/bloomz-560m"
tokenizer_name_or_path = "bigscience/bloomz-560m"
peft_config = PromptTuningConfig(
    task_type=TaskType.CAUSAL_LM,
    prompt_tuning_init=PromptTuningInit.TEXT,
    num_virtual_tokens=8,
    prompt_tuning_init_text="Classify if the tweet is a complaint or not:",
    tokenizer_name_or_path=model_name_or_path,
)

dataset_name = "twitter_complaints"
checkpoint_name = f"{dataset_name}_{model_name_or_path}_{peft_config.peft_type}_{peft_config.task_type}_v1.pt".replace(
    "/", "_"
)
text_column = "Tweet text"
label_column = "text_label"
max_length = 64
lr = 3e-2
num_epochs = 50
batch_size = 8

In [ ]:
dataset = load_dataset("ought/raft", dataset_name)
dataset["train"][0]

Extracting data files:   0%|          | 0/11 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/50 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3399 [00:00<?, ? examples/s]

Dataset raft downloaded and prepared to /root/.cache/huggingface/datasets/ought___raft/twitter_complaints/1.1.0/79c4de1312c1e3730043f7db07179c914f48403101f7124e2fe336f6f54d9f84. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

{'Tweet text': '@HMRCcustomers No this is my first job', 'ID': 0, 'Label': 2}

In [ ]:
dataset["train"].feature

Dataset({
    features: ['Tweet text', 'ID', 'Label'],
    num_rows: 50
})

In [ ]:
dataset["train"].features["Label"].names

['Unlabeled', 'complaint', 'no complaint']

In [ ]:
classes = [k.replace("_", " ") for k in dataset["train"].features["Label"].names]
dataset = dataset.map(
    lambda x: {"text_label": [classes[label] for label in x["Label"]]},
    batched=True,
    num_proc=1,
)
dataset["train"][0]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/3399 [00:00<?, ? examples/s]

{'Tweet text': '@HMRCcustomers No this is my first job',
 'ID': 0,
 'Label': 2,
 'text_label': 'no complaint'}

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

In [ ]:
tokenizer.eos_token_id

2

In [ ]:
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

target_max_length = max([len(tokenizer(class_label)["input_ids"]) for class_label in classes])

print(target_max_length)


3


In [ ]:
import torch

def preprocess_function(examples):
    batch_size = len(examples[text_column])
    inputs = [f"{text_column} : {x} Label : " for x in examples[text_column]]
    targets = [str(x) for x in examples[label_column]]
    model_inputs = tokenizer(inputs)
    labels = tokenizer(targets)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i] + [tokenizer.pad_token_id]
        # print(i, sample_input_ids, label_input_ids)
        model_inputs["input_ids"][i] = sample_input_ids + label_input_ids
        labels["input_ids"][i] = [-100] * len(sample_input_ids) + label_input_ids
        model_inputs["attention_mask"][i] = [1] * len(model_inputs["input_ids"][i])
    # print(model_inputs)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i]
        model_inputs["input_ids"][i] = [tokenizer.pad_token_id] * (
            max_length - len(sample_input_ids)
        ) + sample_input_ids
        model_inputs["attention_mask"][i] = [0] * (max_length - len(sample_input_ids)) + model_inputs[
            "attention_mask"
        ][i]
        labels["input_ids"][i] = [-100] * (max_length - len(sample_input_ids)) + label_input_ids
        model_inputs["input_ids"][i] = torch.tensor(model_inputs["input_ids"][i][:max_length])
        model_inputs["attention_mask"][i] = torch.tensor(model_inputs["attention_mask"][i][:max_length])
        labels["input_ids"][i] = torch.tensor(labels["input_ids"][i][:max_length])
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
processed_datasets = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=dataset["train"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

Running tokenizer on dataset:   0%|          | 0/50 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/3399 [00:00<?, ? examples/s]

In [ ]:
train_dataset = processed_datasets["train"]
eval_dataset = processed_datasets["train"]


train_dataloader = DataLoader(train_dataset, shuffle=True,
                              collate_fn=default_data_collator,
                              batch_size=batch_size,
                              pin_memory=True)
eval_dataloader = DataLoader(eval_dataset,
                             collate_fn=default_data_collator,
                             batch_size=batch_size,
                             pin_memory=True)

In [ ]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(model_name_or_path)

model = get_peft_model(model, peft_config)

print(model.print_trainable_parameters())

trainable params: 8,192 || all params: 559,222,784 || trainable%: 0.0014648902430985358
None


In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataloader) * num_epochs),
)

In [ ]:
model = model.to(device)

In [ ]:
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.detach().float()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    model.eval()
    eval_loss = 0
    eval_preds = []
    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        eval_loss += loss.detach().float()
        eval_preds.extend(
            tokenizer.batch_decode(torch.argmax(outputs.logits, -1).detach().cpu().numpy(), skip_special_tokens=True)
        )

    eval_epoch_loss = eval_loss / len(eval_dataloader)
    eval_ppl = torch.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / len(train_dataloader)
    train_ppl = torch.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
peft_model_id = "Kamaljp/bloomz-560m_PROMPT_TUNING_CAUSAL_LM"
model.push_to_hub("Kamaljp/bloomz-560m_PROMPT_TUNING_CAUSAL_LM", use_auth_token=True)

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.bin:   0%|          | 0.00/33.5k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Kamaljp/bloomz-560m_PROMPT_TUNING_CAUSAL_LM/commit/7a7a82d99c176ec88a2ed91dbef9004e269ce9b5', commit_message='Upload model', commit_description='', oid='7a7a82d99c176ec88a2ed91dbef9004e269ce9b5', pr_url=None, pr_revision=None, pr_num=None)